# to do：
- 回顾之前写的划分MFD的notebook，添加定量分析的函数（CHindex（在写好的部分论文里），NSk，TV）。
- 选出数据，喂到这个notebook里
- adaptation loss前面的lambda用两个类之间的KL散度
- boundary adjustment的时候考虑双向，尽量提供错位的数据，即两个方向可能属于不同类

- 寻找reinforcement learning用于CAV的算法

In [1]:
import time
import math
import tensorflow as tf
import keras 
import numpy as np
import pandas as pd
import ipdb
from keras.models import load_model,Model
from keras.engine.topology import Layer

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
# 定义融合层，将深度学习算法与历史均值算法融合
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [3]:
#定义精度评价指标。为防止0值附近相对误差过大而导致的异常，定义mask层。
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

## 把速度矩阵分类

In [4]:
randseed = 3
class1 = 0
class2 = 1
v = pd.read_csv('./data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
det_with_class = pd.read_csv('./res/%i_id_402_withclass.csv'%randseed, index_col=0)

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = det_with_class[det_with_class['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

v_class1 = v[v['class_i']==class1]
v_class2 = v[v['class_i']==class2]

## 制作 nearest_road_id.csv 和speed.csv

In [5]:
dist_mat = pd.read_csv('./data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('./data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])

## 以上做好了near_road矩阵，接下来做flow/speed矩阵

In [6]:
def get_node(det, seg):
    # det is one single detector id
    # node is one single node id
    
    # seg = pd.read_csv('./data/segement.csv', header=None)
    try:
        node_info = seg[seg[6]==det]
        node = node_info.iloc[0, 0]
    except:
        node_info = seg[seg[7]==det]
        node = node_info.iloc[0, 0]
        
    return node

In [7]:
def get_class_with_node(seg, v_class):
    det_list_class = np.array([])
    try:
        v_class.insert(1, 'id2', '')  # id2 mean node id
    except:
        v_class['id2'] = ''
        
    for i in range(len(v_class)):
        det_list_class = np.append(det_list_class, v_class.iloc[i, 0])
        v_class.iloc[i, 1] = get_node(v_class.iloc[i, 0], seg)
    
    return det_list_class, v_class

In [8]:
def rds_mat(old_dist_mat, det_ids):
    # get a matrix that contains n raods that have specified node id s
    node_ids = np.array([])
    for i in det_ids:
        node_ids = np.append(node_ids, get_node(i, seg))
        
    new_dist_mat = old_dist_mat.loc[node_ids, node_ids]
    old_dist_mat = np.array(old_dist_mat)
    new_near_id_mat = np.argsort(new_dist_mat)
    return new_near_id_mat

In [9]:
seg = pd.read_csv('./data/segement.csv', header=None)

det_list_class1, v_class1 = get_class_with_node(seg, v_class1)
det_list_class2, v_class2 = get_class_with_node(seg, v_class2)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
num_dets = 30

near_road1 = rds_mat(dist_mat, det_list_class1[:num_dets])
near_road2 = rds_mat(dist_mat, det_list_class2[:num_dets])

In [11]:
v_class1 = v_class1[v_class1['id'].isin(det_list_class1[:num_dets])]
v_class2 = v_class2[v_class2['id'].isin(det_list_class2[:num_dets])]

# 源代码如下

In [14]:
print(near_road.shape)
print(flow.shape)

(30, 30)
(30, 5222)


In [14]:
print(near_road.shape)
print(flow.shape)

(30, 30)
(30, 4464)


In [12]:
# near_road = np.array(pd.read_csv('./data/network/2small_network_nearest_road_id.csv',header = 0))
# flow = np.array(pd.read_csv('./data/network/2small_network_speed.csv', header= 0)) #注意header=0 or None
near_road = np.array(near_road1)
flow = np.array(v_class1.iloc[:, 2:-1])

# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 24 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])
        
# ipdb.set_trace()

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前90%数据为训练集，最后10%数据为测试集
image_train_source = image3[:np.shape(image3)[0]*1//10]
image_test_source = image3[np.shape(image3)[0]*1//10:]
label_train_source = label[:np.shape(label)[0]*1//10]
label_test_source = label[np.shape(label)[0]*1//10:]

day_train_source = day[:np.shape(day)[0]*1//10]
day_test_source = day[np.shape(day)[0]*1//10:]


time4 = time.time()
print('input done %g' % (time4-time3))

(3795, 5, 12, 30)
(3795, 30, 3)
(3795, 30, 3)
input done 0.0339088


In [13]:
# near_road = np.array(pd.read_csv('./data/transfer_learning_traffic_data/small_network_nearest_road_id.csv',header = 0))
# flow = np.array(pd.read_csv('./data/transfer_learning_traffic_data/small_network_speed.csv', header= 0)) #注意header=0 or None
near_road = np.array(near_road2)
flow = np.array(v_class2.iloc[:, 2:-1])

# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 24 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train_target = image3[:np.shape(image3)[0]*1//10]
image_test_target = image3[np.shape(image3)[0]*1//10:]
label_train_target = label[:np.shape(label)[0]*1//10]
label_test_target = label[np.shape(label)[0]*1//10:]

day_train_target = day[:np.shape(day)[0]*1//10]
day_test_target = day[np.shape(day)[0]*1//10:]


time4 = time.time()
print('input done %g' % (time4-time3))

(3795, 5, 12, 30)
(3795, 30, 3)
(3795, 30, 3)
input done 0.0368655


In [14]:
#模型构建
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = num_links,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = num_links,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 30)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 12, 30)    120         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 12, 30)    8130        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 5, 12, 30)    0           conv2d_1[0][0]       

In [15]:
#参数加载
finish_model.load_weights('./model/source.h5')


In [16]:
#模型预测
model_pre = finish_model.predict([image_test_target,day_test_target])


In [17]:
#预测结果存储
# model_pre = np.reshape(model_pre,[103, 6])
# model_pre1 = pd.DataFrame(model_pre)
# model_pre1.to_csv('预测值.csv', index = False)

In [18]:
#transfer without FT 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test_source)
smape_mean = smape_loss_func(model_pre, label_test_source)
mae_mean = mae_loss_func(model_pre, label_test_source)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.7802823984459177
smape = 1.3187246744990813
mae = 48.69012338609012


In [20]:

middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_1').output)

In [21]:
middle_result_source = middle.predict([image_train_source, day_train_source])
middle_result_target = middle.predict([image_train_target, day_train_target])

In [22]:
from sklearn import metrics
def mmd (x, y):
    return metrics.mean_squared_error(x,y)

In [23]:
mmd (middle_result_source, middle_result_target)

65.25655

In [24]:
from keras import backend as K

In [25]:
loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
loss2 = 0.001 * mmd (middle_result_source, middle_result_target)
overall_loss = loss1 + loss2

In [26]:
def new_loss(output_final, label_train_target):
    middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_1').output)
    middle_result_source = middle.predict([image_train_source, day_train_source])
    middle_result_target = middle.predict([image_train_target, day_train_target])

    loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
    loss2 = 0.001 * mmd (middle_result_source, middle_result_target)
    overall_loss = loss1 + loss2
    return overall_loss


In [27]:
finish_model.compile(optimizer='adam',loss=new_loss)

In [28]:
finish_model.fit([image_train_target, day_train_target], label_train_target, epochs=100, batch_size=462,
validation_data=([image_test_target,day_test_target], label_test_target))

Train on 379 samples, validate on 3416 samples
Epoch 1/100
379/379 [==============================] - 1s 2ms/step - loss: 1341.7020 - val_loss: 1352.0992
Epoch 2/100
379/379 [==============================] - 0s 268us/step - loss: 1276.1478 - val_loss: 1344.6776
Epoch 3/100
379/379 [==============================] - 0s 263us/step - loss: 1221.7085 - val_loss: 1322.2865
Epoch 4/100
379/379 [==============================] - 0s 242us/step - loss: 1174.1533 - val_loss: 1298.5369
Epoch 5/100
379/379 [==============================] - 0s 261us/step - loss: 1132.3394 - val_loss: 1272.3666
Epoch 6/100
379/379 [==============================] - 0s 245us/step - loss: 1090.9009 - val_loss: 1243.1229
Epoch 7/100
379/379 [==============================] - 0s 248us/step - loss: 1056.8561 - val_loss: 1218.6409
Epoch 8/100
379/379 [==============================] - 0s 250us/step - loss: 1026.5739 - val_loss: 1197.7103
Epoch 9/100
379/379 [==============================] - 0s 250us/step - loss: 988.46

Epoch 77/100
379/379 [==============================] - 0s 279us/step - loss: 264.9536 - val_loss: 186.6098
Epoch 78/100
379/379 [==============================] - 0s 247us/step - loss: 261.9172 - val_loss: 191.1432
Epoch 79/100
379/379 [==============================] - 0s 247us/step - loss: 257.1476 - val_loss: 196.9426
Epoch 80/100
379/379 [==============================] - 0s 247us/step - loss: 253.2394 - val_loss: 202.0345
Epoch 81/100
379/379 [==============================] - 0s 242us/step - loss: 251.0898 - val_loss: 205.3727
Epoch 82/100
379/379 [==============================] - 0s 250us/step - loss: 247.9902 - val_loss: 207.1146
Epoch 83/100
379/379 [==============================] - 0s 234us/step - loss: 246.5869 - val_loss: 207.6514
Epoch 84/100
379/379 [==============================] - 0s 247us/step - loss: 243.0501 - val_loss: 206.0619
Epoch 85/100
379/379 [==============================] - 0s 253us/step - loss: 240.5117 - val_loss: 203.1828
Epoch 86/100
379/379 [======

In [29]:
model_pre = finish_model.predict([image_test_target,day_test_target])

In [30]:
#transfer with DAN 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test_target)
smape_mean = smape_loss_func(model_pre, label_test_target)
mae_mean = mae_loss_func(model_pre, label_test_target)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.20616780639896592
smape = 0.15862540708755657
mae = 8.10429626682493


In [31]:
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test_target[:,i,:])
    mape_list.append(a1)

mape_pd = pd.Series(mape_list)
mape_pd.sort_values()

23    0.058607
12    0.061040
16    0.067711
6     0.086727
25    0.086836
29    0.099014
13    0.105923
4     0.106769
8     0.108234
15    0.116234
19    0.116340
27    0.127481
9     0.130025
28    0.143937
22    0.158643
1     0.159629
10    0.172203
11    0.177028
18    0.199516
14    0.199549
7     0.204625
3     0.231475
24    0.283368
21    0.287563
20    0.326691
2     0.335909
0     0.377245
26    0.440740
17    0.571519
5     0.644691
dtype: float64